In [1]:
import evebox as box
import pandas as pd
import tensorflow as tf
import cProfile
from tqdm.notebook import tqdm, trange

In [2]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.5).with_market_types()

In [3]:
#orders = box.load_orders(uni, tqdm = tqdm)
orders = pd.read_csv('market/current.csv')

In [4]:
#orders.to_csv('market/current.csv')

In [4]:
# Filter all buy orders which are 50x above the median
median   = orders.groupby('type_id')[['price']].median().rename(columns = {'price' : 'median_price'})

orders = orders[
    (50 * orders.merge(median, on = 'type_id')['median_price'] >= orders['price']) |
    ~orders['is_buy_order']
]

In [5]:
# Filter all orders that can not be handled profitably
max_buy  = orders[ orders['is_buy_order']].groupby('type_id')[['price']].max().rename(columns = {'price' : 'max_buy'})
min_sell = orders[~orders['is_buy_order']].groupby('type_id')[['price']].min().rename(columns = {'price' : 'min_sell'})

orders = orders.merge(max_buy, on = 'type_id').merge(min_sell, on = 'type_id')

orders = orders[
    (
        # Sell orders must be below max_buy
        orders['is_buy_order'] |
        (orders['price'] <= orders['max_buy'])
    ) & 
    (
        # Buy orders must be above min_sell
        ~orders['is_buy_order'] |
        (orders['price'] >= orders['min_sell'])
    )
]

In [8]:
# Reduce universe types to all types appearing in the order list

# Pre-filter types
def check_type(t):
    if t["volume"] > 155000:
        return False
    
    return t['type_id'] in orders['type_id']

# Reduce universe
types = [t for t, v in tqdm(uni.types.items()) if check_type(v)]
print('Reducing from {} to {} types'.format(len(uni.types), len(types)))
uni = uni.with_types(types)

# Reduce orders
orders = orders[orders['type_id'].isin(uni.types)]


Reducing from 4606 to 2405 types


In [20]:
orders[orders['type_id'] == 334]

,order_id,duration,is_buy_order,issued,location_id,min_volume,price,range,system_id,type_id,volume_remain,volume_total,max_buy,min_sell


In [8]:
uni.types[628]

{'capacity': 345.0,
 'description': "The Arbitrator is unusual for Amarr ships in that it's primarily a drone carrier. While it is not the best carrier around, it has superior armor that gives it greater durability than most ships in its class.",
 'dogma_attributes': [{'attribute_id': 1547, 'value': 2.0},
  {'attribute_id': 129, 'value': 450.0},
  {'attribute_id': 1154, 'value': 3.0},
  {'attribute_id': 3, 'value': 0.0},
  {'attribute_id': 4, 'value': 11200000.0},
  {'attribute_id': 662, 'value': 0.25},
  {'attribute_id': 263, 'value': 1100.0},
  {'attribute_id': 136, 'value': 1.0},
  {'attribute_id': 9, 'value': 1600.0},
  {'attribute_id': 11, 'value': 695.0},
  {'attribute_id': 12, 'value': 5.0},
  {'attribute_id': 13, 'value': 4.0},
  {'attribute_id': 14, 'value': 4.0},
  {'attribute_id': 15, 'value': 0.0},
  {'attribute_id': 272, 'value': 0.5},
  {'attribute_id': 273, 'value': 0.6},
  {'attribute_id': 274, 'value': 0.8},
  {'attribute_id': 19, 'value': 1.0},
  {'attribute_id': 269,

In [19]:
orders[(orders['system_id'] == 30000142) & ~orders['is_buy_order']][['type_id', 'price', 'volume_remain']].groupby('type_id').aggregate(['sum', 'max', 'min', 'count'])

price                               volume_remain             \
                sum         max         min count           sum        max   
type_id                                                                      
18           503.92       85.00       65.43     7        387617     126883   
21          6573.66      580.59      473.81    13         50121      22630   
37           314.88       22.80       22.31    14     361497719  185953199   
38         15747.90      458.89      401.88    37      37815181    7288100   
39         26400.26      839.63      607.96    41      62895616   20209693   
...             ...         ...         ...   ...           ...        ...   
41266      87115.00    87115.00    87115.00     1             0          0   
41305       3860.34     3860.34     3860.34     1             0          0   
41307       6030.01     6030.01     6030.01     1             0          0   
41312      32109.44    32109.44    32109.44     1             0          0   
43544    8599995.73  4299997.87  4299997.86     2            23         22   

                    
         min count  
type_id             
18        18     7  
21        10    13  
37         3    14  
38       505    37  
39         3    41  
...      ...   ...  
41266      0     1  
41305      0     1  
41307      0     1  
41312      0     1  
43544      1     2  

[145 rows x 8 columns]